<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [37]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


And then we import our movie database

In [48]:
%sql sqlite:///movies.sqlite

'Connected: @movies.sqlite'

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [39]:
%%sql
PRAGMA foreign_keys=ON;

   sqlite:///movies.db
 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [56]:
%%sql
SELECT *
FROM performances

   sqlite:///movies.db
 * sqlite:///movies.sqlite
Done.


performance_nbr,start_time,imdb_nbr,t_name,start_date
94,19:30,tt5580390,Kino,2019-02-23
None,12:23,tt5580390,Kino,2019-02-23


+ Show the performance dates for one of the movies.

In [55]:
%%sql

INSERT
INTO performances(imdb_nbr, t_name, start_date, start_time)
VALUES('tt5580390','Kino','2019-02-23', '12:23');

   sqlite:///movies.db
 * sqlite:///movies.sqlite
1 rows affected.


[]

+ Show all data concerning performances at a given theatere
  on a given date.

In [30]:
%%sql
SELECT theaters.*, performances.*
FROM theaters, performances
WHERE theaters.t_name = performances.t_name
AND theaters.t_name = 'idet'
AND performances.start_date = '2020-02-02'

 * sqlite:///movies.db
   sqlite:///movies.sqlite
(sqlite3.OperationalError) no such table: theaters
[SQL: SELECT theaters.*, performances.*
FROM theaters, performances
WHERE theaters.t_name = performances.t_name
AND theaters.t_name = 'idet'
AND performances.start_date = '2020-02-02']
(Background on this error at: http://sqlalche.me/e/e3q8)


+ List all customers

In [31]:
%%sql
SELECT *
FROM customers

 * sqlite:///movies.db
   sqlite:///movies.sqlite
Done.


user_name,full_name,password
alice,Alice,5dae7886bbe2b1fb134e79f4b83612a16f5b26ef2ac06a91cd7caee0ee7b5557
bob,Bob,053fc66a6cef5cac353d70925c660075075b3be0abc34e01a15ccfc8e17fbcbd


+ List all tickets

In [32]:
%%sql
SELECT *
FROM tickets

 * sqlite:///movies.db
   sqlite:///movies.sqlite
Done.


t_id,p_id,user_name
ee66355e8b19a59982434b0854530cb6,56aaa0779bf585c4a92e12eed9834a6b,alice
40519c5e8a1a9553e07a130940cc866a,56aaa0779bf585c4a92e12eed9834a6b,alice
eb78400168795a7dcba26f37662448dd,56aaa0779bf585c4a92e12eed9834a6b,alice
21ab22df522d000ee012b7753befa6b0,56aaa0779bf585c4a92e12eed9834a6b,alice
853cecd9e71f6a052ac71134911f60a8,56aaa0779bf585c4a92e12eed9834a6b,alice
9dc2b9fffaca54e5f06e5059673d213f,56aaa0779bf585c4a92e12eed9834a6b,alice
fdc27687bae4414c179a6797a4ac0d76,56aaa0779bf585c4a92e12eed9834a6b,alice
2489599f95fbc81fc0698c232a91b1b5,56aaa0779bf585c4a92e12eed9834a6b,alice
050b2751f957d06eff742f3d4b5901e5,56aaa0779bf585c4a92e12eed9834a6b,alice
1b80d7e6677482770c7cfc09726aec11,56aaa0779bf585c4a92e12eed9834a6b,alice


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [33]:
%%sql
INSERT
INTO tickets (performance_nbr, username)
VALUES (3, 'hackerman1337')


 * sqlite:///movies.db
   sqlite:///movies.sqlite
(sqlite3.OperationalError) table tickets has no column named performance_nbr
[SQL: INSERT
INTO tickets (performance_nbr, username)
VALUES (3, 'hackerman1337')]
(Background on this error at: http://sqlalche.me/e/e3q8)


In
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html)
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


In SQLite3 (as of January 2020), we can't do that, instead
  we can use the following idea: each row in a SQLite3 table
  has a `rowid` attribute, it is a unique integer which
  essentially tells in which order the rows were inserted,
  and it's not displayed in queries unless we ask for it.
  SQLite3 also have a function, `last_insert_rowid()`, which
  returns the `rowid` of the last inserted row of a table,
  so we can see the `s_id` of the most recently inserted
  student with the following query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [34]:
%%sql
SELECT id
FROM tickets
WHERE username = 'hackerman1337'

 * sqlite:///movies.db
   sqlite:///movies.sqlite
(sqlite3.OperationalError) no such column: id
[SQL: SELECT id
FROM tickets
WHERE username = 'hackerman1337']
(Background on this error at: http://sqlalche.me/e/e3q8)


+ Try to insert two movie theaters with the same name (this
  should fail).

In [35]:
%%sql
INSERT
INTO theaters
VALUES ('e:a', 120), ('e:a',140);

 * sqlite:///movies.db
   sqlite:///movies.sqlite
(sqlite3.OperationalError) no such table: theaters
[SQL: INSERT
INTO theaters
VALUES ('e:a', 120), ('e:a',140);]
(Background on this error at: http://sqlalche.me/e/e3q8)


+ Try to insert a performance where the theater doesnâ€™t
  exist in the database (this should fail).

In [36]:
%%sql
INSERT 
INTO performances
VALUES 
(1, '17:30:00', 6584, 'edekvata', '2020-02-02')

 * sqlite:///movies.db
   sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT 
INTO performances
VALUES 
(1, '17:30:00', 6584, 'edekvata', '2020-02-02')]
(Background on this error at: http://sqlalche.me/e/gkpj)

+ Create a ticket where either the user or the performance
  doesnâ€™t exist.

In [ ]:
%%sql
INSERT INTO tickets (username, performance_nbr)
VALUES
('sverre', 19);